# Pre Processing

## Pre Processing 1

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('all-vehicles-model.csv', on_bad_lines='skip', sep=';')
pd.set_option('display.max_columns', None)

In [ ]:
if 'Cylinders' in data.columns:
    data['Cylinders'].fillna(0, inplace=True)

if 'Electric motor' in data.columns:
    data['Electric motor'].fillna('Not Electric', inplace=True)

if 'ATV Type' in data.columns:
    data['ATV Type'].fillna('Petrol', inplace=True)

if 'Drive' in data.columns:
    data['Drive'].fillna('All-Wheel Drive', inplace=True)

In [ ]:
unnecessary_cols = [
    'Annual Petroleum Consumption For Fuel Type1',
    'Annual Petroleum Consumption For Fuel Type2',
    'Time to charge at 120V',
    'Time to charge at 240V',
    'Unrounded City Mpg For Fuel Type1 (2)',
    'Unrounded City Mpg For Fuel Type2',
    'City gasoline consumption',
    'City electricity consumption',
    'EPA city utility factor',
    'Co2 Fuel Type2',
    'Co2  Tailpipe For Fuel Type2',
    'Co2  Tailpipe For Fuel Type1',
    'Combined Mpg For Fuel Type1',
    'Unrounded Combined Mpg For Fuel Type1',
    'Combined Mpg For Fuel Type2',
    'Unrounded Combined Mpg For Fuel Type2',
    'Combined electricity consumption',
    'Combined gasoline consumption',
    'EPA combined utility factor',
    'Engine displacement',
    'EPA model type index',
    'EPA Fuel Economy Score',
    'Annual Fuel Cost For Fuel Type2',
    'GHG Score Alternative Fuel',
    'Highway Mpg For Fuel Type1',
    'Unrounded Highway Mpg For Fuel Type1',
    'Highway Mpg For Fuel Type2',
    'Unrounded Highway Mpg For Fuel Type2',
    'Highway gasoline consumption',
    'Highway electricity consumption',
    'EPA highway utility factor',
    'Hatchback luggage volume',
    'Hatchback passenger volume',
    '2 door luggage volume',
    '4 door luggage volume',
    'MPG Data',
    'PHEV Blended',
    '2-door passenger volume',
    '4-door passenger volume',
    'Range For Fuel Type1',
    'Range  City For Fuel Type1',
    'Range  City For Fuel Type2',
    'Range  Highway For Fuel Type1',
    'Range  Highway For Fuel Type2',
    'Unadjusted City Mpg For Fuel Type1',
    'Unadjusted City Mpg For Fuel Type2',
    'Unadjusted Highway Mpg For Fuel Type1',
    'Unadjusted Highway Mpg For Fuel Type2',
    'You Save/Spend',
    'Guzzler',
    'Transmission descriptor',
    'T Charger',
    'S Charger',
    'Epa Range For Fuel Type2',
    'c240Dscr',
    'charge240b',
    'C240B Dscr',
    'Created On',
    'Modified On',
    'PHEV City',
    'PHEV Highway',
    'PHEV Combined',
    'City Mpg For Fuel Type2',
    'Fuel Type2',
    'MFR Code',
    'Fuel Type1',
    'Start-Stop',
    'Engine descriptor',
    'GHG Score'
]
data.drop(unnecessary_cols, axis=1, inplace=True)

In [ ]:
null_data = data[data.isnull().any(axis=1)]
data.dropna(inplace = True)

## Pre Processing 2

In [ ]:
data['Year'] = pd.to_numeric(data['Year'])

In [ ]:
data.sort_values(by='Year', ascending=False, inplace=True)
data

In [ ]:
data_unique = data.drop_duplicates(subset=['Model'], keep='first')
data_unique.reset_index(drop=True, inplace=True)

## Pre Processing 3

In [ ]:
!pip install -U duckduckgo_search

In [ ]:
from duckduckgo_search import DDGS
cars = []
for i in range(0, 4876):
  car = data["Make"][i] + " " + data["Model"][i] + " " + data["Year"][i].astype(str)
  results = DDGS().images(
      keywords=car,
      region="wt-wt",
      safesearch="off",
      size=None,
      color="color",
      type_image="photo",
      layout="wide",
      license_image=None,
      max_results=1,
  )
  if len(results):
    data.loc[data['Model'] == data["Model"][i], "Image"] = results[0]["image"]
  else:
    cars.append(car)

In [ ]:
data['Year'] = data['Year'].astype(str)

## Pre Processing 4

In [ ]:
import re
from duckduckgo_search import DDGS

cars = []
prices = []

for i in range(1006, 4876):
  car = data["Make"][i] + " " + data["Model"][i] + " " + data["Year"][i].astype(str) + " price in dollars"
  results = DDGS().text(car, max_results=5)
  j = 5
  print(i)
  print(car)
  l = True
  while l:
    for k in range(0, j):
      body_text = results[k]['body']
      matches = re.findall(r'\$[\d,]{4,}(?: - \$[\d,]+)?', body_text)
      if matches:
        print(body_text)
        print(matches[0])
        data.loc[data['Model'] == data["Model"][i], "Price"] = matches[0]
        l = False
        break

    if l:
      if j<=95:
        j=j+5
        results = DDGS().text(car, max_results=j)
      else:
        l=False

In [ ]:
data.to_csv('final_car_data.csv')

# Recommendation Models

## Word Embeddings

In [ ]:
!pip install gensim scikit-learn

In [ ]:
import pandas as pd
data = pd.read_csv('final_car_data.csv', on_bad_lines='skip', sep=',')
pd.set_option('display.max_columns', None)

In [ ]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

In [ ]:
car_features_data = data.astype(str)
car_features_list = data.iloc[:, :].values.tolist()
model = Word2Vec(car_features_list, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec_model_similarity")

In [ ]:
car_features_data = data.astype(str)
car_features_list = data.iloc[:, :].values.tolist()
model = Word2Vec.load("word2vec_model_similarity")

def calculate_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

def get_word_vector(feature):
    return model.wv[feature]

def get_average_vector(features):
    vectors = [get_word_vector(feature) for feature in features if feature in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

car_vectors_list = []

for index, row in car_features_data.iterrows():
    car_features = row.tolist()
    car_vector = get_average_vector(car_features)
    car_vectors_list.append(car_vector)

columns = [f"feature_{i}" for i in range(len(car_vectors_list[0]))]
car_vectors_data = pd.DataFrame(car_vectors_list, columns=columns)
car_vectors_data.to_csv("car_vectors_similarity.csv", index=False)

def calculate_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

car_vectors_data = pd.read_csv('car_vectors_similarity.csv')
user_preference = car_features_list[0]
user_preference = [str(element) for element in user_preference]
user_preference_vector = get_average_vector(user_preference)

car_similarities = {}
for index, row in car_vectors_data.iterrows():
    car_vector = row.tolist()
    similarity = calculate_similarity(user_preference_vector, car_vector)
    car_similarities[index] = {'similarity': similarity}

sorted_cars = sorted(car_similarities.items(), key=lambda x: x[1]['similarity'], reverse=True)

print("Ranked cars based on user preference similarity:")
for car_index, car_info in sorted_cars[:100]:
    print(f"Similarity: {car_info['similarity']:.2f}, Car: {data.iloc[car_index]}")

## K Means

In [ ]:
import pickle
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import pandas as pd
data = pd.read_csv('final_car_data.csv', on_bad_lines='skip', sep=',')
pd.set_option('display.max_columns', None)

In [ ]:
car_data = data
car_data = data.drop(columns=['Make', 'Model', 'baseModel', 'Image', 'Cylinders', 'Drive', 'Fuel Type', 'Transmission', 'Electric motor', 'Annual Fuel Cost For Fuel Type1'])

In [ ]:
car_features_data = car_data.astype(str)
car_features_list = car_data.iloc[:, :].values.tolist()
model = Word2Vec(car_features_list, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec_model_cluster")

In [ ]:
car_features_data = car_data.astype(str)
car_features_list = car_data.iloc[:, :].values.tolist()
model = Word2Vec.load("word2vec_model_cluster")

def calculate_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

def get_word_vector(feature):
    return model.wv[feature]

def get_average_vector(features):
    vectors = [get_word_vector(feature) for feature in features if feature in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

car_vectors_list = []
for index, row in car_features_data.iterrows():
    car_features = row.tolist()
    car_vector = get_average_vector(car_features)
    car_vectors_list.append(car_vector)

columns = [f"feature_{i}" for i in range(len(car_vectors_list[0]))]
car_vectors_data = pd.DataFrame(car_vectors_list, columns=columns)
car_vectors_data.to_csv("car_vectors_cluster.csv", index=False)

In [ ]:
car_vectors_data = pd.read_csv("car_vectors_cluster.csv")

scaler = MinMaxScaler()
car_vectors_data = pd.DataFrame(scaler.fit_transform(car_vectors_data), columns=car_vectors_data.columns)

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(car_vectors_data)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.xticks(np.arange(1, 11, 1))
plt.grid(True)
plt.show()

In [ ]:
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(car_vectors_data)

In [ ]:
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

data['cluster_label'] = cluster_labels
car_vectors_data['cluster_label'] = cluster_labels

data.to_csv("car_clusters.csv")
car_vectors_data.to_csv("car_vectors_cluster_label.csv")

In [ ]:
data

In [ ]:
import joblib
joblib.dump(scaler, 'minmax_scaler.pkl')
scaler = joblib.load('minmax_scaler.pkl')

In [ ]:
with open('kmeans_model.pkl', 'rb') as f:
    loaded_kmeans = pickle.load(f)

In [ ]:
data_user_preference = {
  'City Mpg For Fuel Type1': ["20"],
  'Co2 Fuel Type1': ["400"],
  'Vehicle Size Class': ["Standard Pickup Trucks 4WD"],
  'Year': ["2024"],
  'ATV Type': ["Petrol"],
  'Price': ["$40,000"]
}
df = pd.DataFrame(data_user_preference)

centroid_vectors = loaded_kmeans.cluster_centers_
user_preference_vector = get_average_vector(df.iloc[0])
user_preference_vector = pd.DataFrame(user_preference_vector).T
user_preference_vector.columns = [f'feature_{i}' for i in range(len(user_preference_vector.columns))]
user_preference_vector = pd.DataFrame(scaler.transform(user_preference_vector), columns = user_preference_vector.columns)

distances = pairwise_distances(user_preference_vector, centroid_vectors, metric='euclidean')
closest_center_index = np.argmin(distances)

print(distances)
print(f"User preference is closest to Cluster {closest_center_index}")

In [ ]:
car_vectors_cluster_label = pd.read_csv('car_vectors_cluster_label.csv')

similiar_cars_vectors = car_vectors_cluster_label.loc[car_vectors_cluster_label['cluster_label'] == closest_center_index]
car_data = similiar_cars_vectors.drop(columns=['Unnamed: 0', 'cluster_label'])

user_preference = user_preference_vector.values.squeeze()

def calculate_similarity(vec1, vec2):
  return cosine_similarity([vec1], [vec2])[0][0]

car_similarities = {}
for index, row in car_data.iterrows():
  car_vector = row.values
  similarity = calculate_similarity(user_preference, car_vector)
  car_similarities[index] = {'similarity': similarity}

sorted_cars = sorted(car_similarities.items(), key=lambda x: x[1]['similarity'], reverse=True)
print(sorted_cars)

In [ ]:
data.iloc[2479]

## Matrix Factorization

### Generating User Data

In [ ]:
car_data = pd.read_csv("car_clusters.csv")
car_data.head()

,Unnamed: 0,Make,Model,City Mpg For Fuel Type1,Co2 Fuel Type1,Cylinders,Drive,Annual Fuel Cost For Fuel Type1,Fuel Type,ID,Transmission,Vehicle Size Class,Year,ATV Type,Electric motor,baseModel,Image,Price,cluster_label
0,0,Chevrolet,Silverado 4WD,18,468,4.0,4-Wheel Drive,3050,Regular,47122,Automatic 8-spd,Standard Pickup Trucks 4WD,2024,Petrol,Not Electric,Silverado,https://hips.hearstapps.com/hmg-prod/images/20...,"$43,487",1
1,1,Alfa Romeo,Stelvio AWD,22,364,4.0,All-Wheel Drive,3000,Premium,46596,Automatic 8-spd,Small Sport Utility Vehicle 4WD,2024,Petrol,Not Electric,Stelvio,https://www.motortrend.com/uploads/2022/11/202...,"$47,545",1
2,2,BMW,Z4 sDrive30i,25,312,4.0,Rear-Wheel Drive,2550,Premium,46631,Automatic (S8),Two Seaters,2024,Petrol,Not Electric,Z4,https://cars.chapmanchoice.com/stock/bmw/20991...,"$56,977",3
3,3,Genesis,G80 AWD,22,361,4.0,All-Wheel Drive,2850,Premium,46671,Automatic (S8),Large Cars,2024,Petrol,Not Electric,G80,https://media.ed.edmunds-media.com/genesis/g80...,"$55,650",3
4,4,BMW,X4 xDrive30i,21,369,4.0,All-Wheel Drive,3000,Premium,46705,Automatic (S8),Small Sport Utility Vehicle 4WD,2024,Petrol,Not Electric,X4,https://cdn.carbuzz.com/car-thumbnails/700x375...,"$55,000",3


In [ ]:
car_data = car_data.rename(columns={'Unnamed: 0': 'Index'})
car_data.head()

,Index,Make,Model,City Mpg For Fuel Type1,Co2 Fuel Type1,Cylinders,Drive,Annual Fuel Cost For Fuel Type1,Fuel Type,ID,Transmission,Vehicle Size Class,Year,ATV Type,Electric motor,baseModel,Image,Price,cluster_label
0,0,Chevrolet,Silverado 4WD,18,468,4.0,4-Wheel Drive,3050,Regular,47122,Automatic 8-spd,Standard Pickup Trucks 4WD,2024,Petrol,Not Electric,Silverado,https://hips.hearstapps.com/hmg-prod/images/20...,"$43,487",1
1,1,Alfa Romeo,Stelvio AWD,22,364,4.0,All-Wheel Drive,3000,Premium,46596,Automatic 8-spd,Small Sport Utility Vehicle 4WD,2024,Petrol,Not Electric,Stelvio,https://www.motortrend.com/uploads/2022/11/202...,"$47,545",1
2,2,BMW,Z4 sDrive30i,25,312,4.0,Rear-Wheel Drive,2550,Premium,46631,Automatic (S8),Two Seaters,2024,Petrol,Not Electric,Z4,https://cars.chapmanchoice.com/stock/bmw/20991...,"$56,977",3
3,3,Genesis,G80 AWD,22,361,4.0,All-Wheel Drive,2850,Premium,46671,Automatic (S8),Large Cars,2024,Petrol,Not Electric,G80,https://media.ed.edmunds-media.com/genesis/g80...,"$55,650",3
4,4,BMW,X4 xDrive30i,21,369,4.0,All-Wheel Drive,3000,Premium,46705,Automatic (S8),Small Sport Utility Vehicle 4WD,2024,Petrol,Not Electric,X4,https://cdn.carbuzz.com/car-thumbnails/700x375...,"$55,000",3


In [ ]:
num_users = 610  # Total number of users
user_ids = range(0, num_users + 1)
user_data = pd.DataFrame(columns=['userId', 'carId', 'rating'])

for user_id in user_ids:
    user_specific_cluster = np.random.randint(0, 4)

    cluster_weights = {0: np.random.uniform(0.1, 0.3),
                       1: np.random.uniform(0.1, 0.3),
                       2: np.random.uniform(0.1, 0.3),
                       3: np.random.uniform(0.1, 0.3)}

    cluster_weights[user_specific_cluster] *= 25

    cluster_probs = np.array([cluster_weights[cluster] for cluster in car_data['cluster_label']])

    num_entries = np.random.randint(5, 100)
    sampled_cars = np.random.choice(car_data['Index'], size=num_entries, p=cluster_probs / np.sum(cluster_probs))

    ratings = []
    for car_id in sampled_cars:
        if car_data.loc[car_data['Index'] == car_id, 'cluster_label'].iloc[0] == user_specific_cluster:
            rating = np.random.choice([3, 4, 5], p=[0.5, 0.2, 0.3])
        else:
            rating = np.random.choice([1, 2], p=[0.6, 0.4])
        ratings.append(rating)

    user_data = pd.concat([user_data, pd.DataFrame({'userId': [user_id] * num_entries,
                                                    'carId': sampled_cars,
                                                    'rating': ratings})], ignore_index=True)

user_data = user_data.sample(frac=1).reset_index(drop=True)
print(user_data.head())

  userId carId rating
0    369   435      3
1     56  4353      3
2    531  2017      3
3     25  3643      3
4    192  3038      5


In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31576 entries, 0 to 31575
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userId  31576 non-null  object
 1   carId   31576 non-null  object
 2   rating  31576 non-null  object
dtypes: object(3)
memory usage: 740.2+ KB


In [ ]:
user_data = user_data.sort_values(by="userId")

def count_values(df):
    counts = {}
    for column in df.columns:
        counts[column] = df[column].value_counts()
    return counts

counts = count_values(user_data)
print(counts)

{'userId': userId
324    99
102    99
469    99
411    99
564    98
       ..
321     5
315     5
266     5
221     5
223     5
Name: count, Length: 611, dtype: int64, 'carId': carId
888     29
3654    29
4669    27
1410    25
3030    24
        ..
2854     1
445      1
2460     1
4401     1
2838     1
Name: count, Length: 4841, dtype: int64, 'rating': rating
3    13253
5     7837
4     5254
1     3130
2     2102
Name: count, dtype: int64}


In [ ]:
user_data.head()
user_data.to_csv('ratings.csv', index = False)

### Model

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model
from keras.utils import plot_model

from scipy.sparse import vstack
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [ ]:
car_ratings = pd.read_csv('ratings.csv')
print('Shape Car-ratings:\t{}'.format(car_ratings.shape))
car_ratings.head()

Shape Car-ratings:	(31576, 3)


,userId,carId,rating
0,0,2409,4
1,0,435,3
2,0,2210,3
3,0,2117,3
4,0,38,5


In [ ]:
df_p = car_ratings.pivot_table(index='userId', columns='carId', values='rating')
df_p.sample(6)

carId,0,1,2,3,4,5,6,7,8,9,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
userId,,,,,,,,,,,,,,,,,,,,,
216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = car_ratings
user_ids = df["userId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
car_ids = df["carId"].unique().tolist()
car2car_encoded = {x: i for i, x in enumerate(car_ids)}
car_encoded2car = {i: x for i, x in enumerate(car_ids)}
df["user"] = df["userId"].map(user2user_encoded)
df["car"] = df["carId"].map(car2car_encoded)

num_users = len(user2user_encoded)
num_cars = len(car_encoded2car)

df = df.sample(frac=1, random_state=42)
x = df[["user", "car"]].values
y = df["rating"].apply(lambda x: (x - 0.5) / (4.5)).values

train_indices = int(0.9 * df.shape[0])

x_train, x_val, x_test, y_train, y_val, y_test = (
    x[:train_indices],
    x[train_indices:-1000],
    x[-1000:],
    y[:train_indices],
    y[train_indices:-1000],
    y[-1000:],
)

In [ ]:
embedding_size= 50

user_id_input = Input(shape=[1], name='user')
car_id_input = Input(shape=[1], name='car')

user_embedding = Embedding(output_dim=embedding_size,
                           input_dim=num_users,
                           input_length=1,
                           embeddings_initializer="he_normal",
                           embeddings_regularizer=keras.regularizers.l2(1e-6),
                           name='user_embedding')(user_id_input)
car_embedding = Embedding(output_dim=embedding_size,
                            input_dim=num_cars,
                            input_length=1,
                            embeddings_initializer="he_normal",
                            embeddings_regularizer=keras.regularizers.l2(1e-6),
                            name='car_embedding')(car_id_input)

user_vector = Reshape([embedding_size])(user_embedding)
car_vector = Reshape([embedding_size])(car_embedding)

concat = Concatenate()([user_vector, car_vector])
dense1 = Dense(256)(concat)
dense = Dropout(0.2)(dense1)
y = Dense(1, activation="sigmoid")(dense)

model = Model(inputs=[user_id_input, car_id_input], outputs=y)
model.compile(loss='mse',  optimizer = 'adam')

history = model.fit(
            x = [x_train[:,0],x_train[:,1]],
            y = y_train,
            batch_size=256,
            epochs=4,
            validation_data = ([x_val[:,0], x_val[:,1]], y_val),
          )

Epoch 1/4
112/112 [==============================] - 3s 14ms/step - loss: 0.0722 - val_loss: 0.0668
Epoch 2/4
112/112 [==============================] - 1s 9ms/step - loss: 0.0598 - val_loss: 0.0655
Epoch 3/4
112/112 [==============================] - 1s 7ms/step - loss: 0.0531 - val_loss: 0.0646
Epoch 4/4
112/112 [==============================] - 1s 7ms/step - loss: 0.0504 - val_loss: 0.0655


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 car (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 50)                30550     ['user[0][0]']                
                                                                                                  
 car_embedding (Embedding)   (None, 1, 50)                242050    ['car[0][0]']                 
                                                                                              

In [ ]:
y_pred = model.predict([x_test[:,0], x_test[:,1]])
y_true =  y_test
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result : {:.4f} RMSE'.format(rmse))

32/32 [==============================] - 0s 1ms/step


Testing Result : 0.2449 RMSE


In [ ]:
p , a = (model.predict([x_test[:15,0], x_test[:15,1]]) , y_test[:15])
print("Ratings are Normalized between 0 and 1 (1 - 5)")
for i in range(len(p)):
    print("Predicted rating is : "+ str(np.round(p[i],2)) +"        Actual rating was : "+ str(round(a[i],2)) )

1/1 [==============================] - 0s 36ms/step
Ratings are Normalized between 0 and 1 (1 - 5)
Predicted rating is : [0.76]        Actual rating was : 0.78
Predicted rating is : [0.66]        Actual rating was : 0.78
Predicted rating is : [0.7]        Actual rating was : 0.56
Predicted rating is : [0.67]        Actual rating was : 1.0
Predicted rating is : [0.71]        Actual rating was : 0.56
Predicted rating is : [0.67]        Actual rating was : 1.0
Predicted rating is : [0.41]        Actual rating was : 0.56
Predicted rating is : [0.84]        Actual rating was : 1.0
Predicted rating is : [0.71]        Actual rating was : 1.0
Predicted rating is : [0.64]        Actual rating was : 0.56
Predicted rating is : [0.66]        Actual rating was : 1.0
Predicted rating is : [0.72]        Actual rating was : 0.56
Predicted rating is : [0.66]        Actual rating was : 0.56
Predicted rating is : [0.71]        Actual rating was : 1.0
Predicted rating is : [0.59]        Actual rating was 

In [ ]:
car_df = pd.read_csv('final_car_data.csv')
car_df['carId'] = car_df.index

def get_recommendations(user_id):
      cars_watched_by_user = df[df.userId == user_id]
      cars_not_watched = car_df[
          ~car_df["carId"].isin(cars_watched_by_user.carId.values)]["carId"]
      cars_not_watched = list(
          set(cars_not_watched).intersection(set(car2car_encoded.keys()))
      )
      cars_not_watched = [[car2car_encoded.get(x)] for x in cars_not_watched]
      user_encoder = user2user_encoded.get(user_id)
      user_car_array = np.hstack(
          ([[user_id]] * len(cars_not_watched), cars_not_watched)
      )

      ratings = model.predict([user_car_array[:,0], user_car_array[:,1]]).flatten()

      top_ratings_indices = ratings.argsort()[-10:][::-1]
      recommended_car_ids = [
          car_encoded2car.get(cars_not_watched[x][0]) for x in top_ratings_indices
      ]

      print("Showing recommendations for user: {}".format(user_id))
      print("=" * 36)
      print("Cars with high ratings from user")
      print("-" * 34)
      top_cars_user = (
          cars_watched_by_user.sort_values(by="rating", ascending=False)
          .head(5)
          .carId.values
      )
      car_df_rows = car_df[car_df["carId"].isin(top_cars_user)]
      for row in car_df_rows.itertuples():
          print(row)

      print("-" * 30)
      print(" Top 10 car recommendations")
      print("-" * 30)
      recommended_cars = car_df[car_df["carId"].isin(recommended_car_ids)]
      for row in recommended_cars.itertuples():
          print(row)

In [ ]:
get_recommendations(3)

151/151 [==============================] - 0s 1ms/step
Showing recommendations for user: 3
Cars with high ratings from user
----------------------------------
Pandas(Index=108, Make='Porsche', Model='Cayenne S', _3=15, _4=521, Cylinders=8.0, Drive='All-Wheel Drive', _7=4200, _8='Premium', ID=46906, Transmission='Automatic (S8)', _11='Standard Sport Utility Vehicle 4WD', Year=2024, _13='Petrol', _14='Not Electric', baseModel='Cayenne', Image='https://images.cars.com/cldstatic/wp-content/uploads/porsche-cayenne-2024-exterior-oem-07.jpg', Price='$80,850', carId=108)
Pandas(Index=3458, Make='Subaru', Model='Legacy/Outback Wagon AWD', _3=19, _4=-1, Cylinders=4.0, Drive='4-Wheel or All-Wheel Drive', _7=2750, _8='Regular', ID=19935, Transmission='Manual 5-spd', _11='Midsize Station Wagons', Year=2004, _13='Petrol', _14='Not Electric', baseModel='Legacy/Outback', Image='https://www.carspecs.us/photos/afb53c4945d4cde262ab68de312c2fe1ed475db9-2000.jpg', Price='$33,045', carId=3458)
Pandas(Index=

In [ ]:
user_id = 611

user_data = pd.DataFrame(columns=['userId', 'carId', 'rating'])

user_specific_cluster = np.random.randint(0, 4)

cluster_weights = {0: np.random.uniform(0.1, 0.3),
                    1: np.random.uniform(0.1, 0.3),
                    2: np.random.uniform(0.1, 0.3),
                    3: np.random.uniform(0.1, 0.3)}

cluster_weights[user_specific_cluster] *= 25

cluster_probs = np.array([cluster_weights[cluster] for cluster in car_data['cluster_label']])

num_entries = np.random.randint(5, 100)
sampled_cars = np.random.choice(car_data['Index'], size=num_entries, p=cluster_probs / np.sum(cluster_probs))

ratings = []
for car_id in sampled_cars:
    if car_data.loc[car_data['Index'] == car_id, 'cluster_label'].iloc[0] == user_specific_cluster:
        rating = np.random.choice([3, 4, 5], p=[0.5, 0.2, 0.3])
    else:
        rating = np.random.choice([1, 2], p=[0.6, 0.4])
    ratings.append(rating)

user_data = pd.concat([user_data, pd.DataFrame({'userId': user_id,
                                                'carId': sampled_cars,
                                                'rating': ratings})], ignore_index=True)

In [ ]:
user_data
user_data.to_csv('user_ratings.csv', index = False)

In [ ]:
existing_df = pd.read_csv('/content/ratings.csv')
new_user_ratings_df = pd.read_csv('/content/user_ratings.csv')
print('Shape Existing-ratings:\t{}'.format(existing_df.shape))
print('Shape New-ratings:\t{}'.format(new_user_ratings_df.shape))

In [ ]:
new_user_ratings_df.head()

NameError: name 'new_user_ratings_df' is not defined

In [ ]:
combined_df = pd.concat([existing_df, new_user_ratings_df], ignore_index=True)
car_ratings = combined_df
print('Shape Car-ratings:\t{}'.format(car_ratings.shape))